In [1]:
import random, numpy

In [2]:
def lagrange_interpolation(x_data: list[float], y_data: list[float], x_int: int) -> list[float]:
    # check to make sure data lengths match
    if(len(x) != len(y)):
        print("Dataset lengths must match")
        return
    n = len(x)

    # define interpolation
    p_interp = [0.0 for _ in range(x_int)]

    for i in range(n):
        L = [1.0 for _ in range(x_int)]
        for j in range(n):
            if(i == j):
                continue
            L[j] *= (x_int - x_data[j]) / (x_data[j] - x_data[i])
        p_interp = [point + y_data[i] * L[i] for point in p_interp]
    return(p_interp)

In [8]:
def N(d: float) -> float: return d * random.gauss(1)

In [4]:
def f_1(x: float) -> float: return(2*x + 4 + N(0.5))
def f_2(x: float) -> float: return(x**3 - 3*x + 1 + N(0.5))

In [11]:
def linear_spline(datx: list[float], daty: list[float], x_interp: int) -> list[float]:
    n = len(datx)
    a = [0.0 for _ in range(n - 1)]
    b = a.copy()
    p_interp = []
    for i in range(n - 1):
        a[i] = (daty[i + 1] - daty[i]) / (datx[i + 1] - datx[i])
        b[i] = -a[i]*datx[i] + daty[i]
    for i in range(n - 1):
        for x in range(datx[i], datx[i+1], datx[i+1] - datx[i] / 100):
            p_interp.append(a[i]*x + b[i])
        # linearly interpolate b/t xs
        # create array of interpolated values
        # append interpolated values to p_interp
    return(p_interp)

In [12]:
xs = [i for i in range(10)]
ys = [f_1(i) for i in range(10)]

In [13]:
linear_spline(xs, ys, 0)

TypeError: 'float' object cannot be interpreted as an integer

In [15]:
[i for i in range(0, 10, 0.01)]

TypeError: 'float' object cannot be interpreted as an integer

In [17]:
arr = ["a", "b"] + ["c", "d"]

In [18]:
arr += "e"

In [19]:
arr

['a', 'b', 'c', 'd', 'e']

In [20]:
1 + [2, 3]

TypeError: unsupported operand type(s) for +: 'int' and 'list'